In [ ]:
! pip install pandas scikit-learn
!pip install -U imbalanced-learn

In [ ]:
! git clone --recursive https://github.com/Microsoft/LightGBM



Cloning into 'LightGBM'...
remote: Enumerating objects: 36260, done.
remote: Counting objects: 100% (4101/4101), done.
remote: Compressing objects: 100% (383/383), done.
remote: Total 36260 (delta 3946), reused 3719 (delta 3718), pack-reused 32159 (from 3)
Receiving objects: 100% (36260/36260), 23.60 MiB | 14.86 MiB/s, done.
Resolving deltas: 100% (27015/27015), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21782, done.        
remote: Counting objects: 100% (19/19), done.   

In [ ]:
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found OpenMP_C: -fopenmp (found version "4.5")
-- Found OpenMP_CXX: -fopenmp (found version "4.5")
-- Found OpenMP: TRUE (found version "4.5")
-- Looking for CL_VERSION_3_0
-- Looking for CL_VERSION_3_0 - found
-- Found OpenCL: /usr/lib/x86_64-linux-gnu/libOpenCL.so (found version "3.0")
-- OpenCL include directory: /usr/include
CMake Warning (dev) at CMakeLists.txt:196 (find_package):
  Policy CMP0167 is not set: The FindBoost module is removed.  Run "cmake
  --help-policy CMP0167" for 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# load training data:
data = pd.read_csv('gdrive/MyDrive/SemEval25/food_recall_incidents.csv', index_col=0)
trainset, devset = train_test_split(data, test_size=0.2, random_state=2024)

trainset.sample()

,product-category,hazard-category
title,,
Instant Reisnudeln Vegetarisch,cereals and bakery products,chemical


In [35]:
from imblearn.pipeline import Pipeline
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from imblearn.over_sampling import RandomOverSampler

from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb

text_clf_lr = Pipeline([
    ('vect', TfidfVectorizer(strip_accents='unicode', analyzer='char', ngram_range=(2,5), max_df=0.5, min_df=5)),
     ('sm', RandomOverSampler(random_state=0)),
     ('clf', lgb.LGBMClassifier(application='multiclass', min_data_in_leaf=20, verbosity=-1, boosting='dart',learning_rate=0.07,max_leaves=1024))
    ])

In [36]:
from sklearn.metrics import classification_report, f1_score

for label in ('hazard-category', 'product-category'):
  print(label.upper())
  text_clf_lr.fit(trainset.title, trainset[label])

  # get development scores:
  devset['predictions-' + label] = text_clf_lr.predict(devset.title)
  print(f'  macro: {f1_score(devset[label], devset["predictions-" + label], zero_division=0, average="macro"):.2f}')
  print(f'  micro: {f1_score(devset[label], devset["predictions-" + label], zero_division=0, average="micro"):.2f}')

  # predict test set:
  #testset[label] = text_clf_lr.predict(testset.title)

HAZARD-CATEGORY


AttributeError: 'DataFrame' object has no attribute 'title'

In [ ]:
def compute_score(hazards_true, products_true, hazards_pred, products_pred):
  # compute f1 for hazards:
  f1_hazards = f1_score(
    hazards_true,
    hazards_pred,
    average='macro'
  )

  # compute f1 for products:
  f1_products = f1_score(
    products_true[hazards_pred == hazards_true],
    products_pred[hazards_pred == hazards_true],
    average='macro'
  )

  return (f1_hazards + f1_products) / 2.

print(f"Score Sub-Task 1: {compute_score(devset['hazard-category'], devset['product-category'], devset['predictions-hazard-category'], devset['predictions-product-category']):.3f}")
#print(f"Score Sub-Task 2: {compute_score(devset['hazard'], devset['product'], devset['predictions-hazard'], devset['predictions-product']):.3f}")

Score Sub-Task 1: 0.515


KeyError: 'predictions-hazard'